# Final CHO Model
This notebook is to asses the validity of our reconstruction and how complete it is.

In [13]:
# Import libraries
import gspread
import pandas as pd
import numpy as np

import cobra
from cobra import Model, Reaction, Metabolite
from cobra.io import validate_sbml_model, write_sbml_model

from tqdm.notebook import tqdm

### 1. Retrieve information from the Google Sheet datasets reactions and metabolites

In [6]:
# give service account details to gspread
sa = gspread.service_account(filename='credentials.json')

# sa is a gspread client, which can be used for connecting to the sheets
# by using the open method and the sheet name.
cho_recon = sa.open('CHO Network Reconstruction')

# we also need to specify the page name before getting the data. In this case we use the Rxns and Metabolites sheet.
rxns_sheet = cho_recon.worksheet('Rxns')
rxns_attributes_sheet = cho_recon.worksheet('Attributes')
metabolites_sheet = cho_recon.worksheet('Metabolites')

# We can extract the data using the get_all_records method and create pd DataFrames

# Reactions IDs, names, formulas, GPRs
rxns = pd.DataFrame(rxns_sheet.get_all_records())

# Reactions bounds
rxns_attributes = pd.DataFrame(rxns_attributes_sheet.get_all_records())

# Metabolites names, formulas and compartment
metabolites = pd.DataFrame(metabolites_sheet.get_all_records())

### 2. Build a model and feed it the information from the df generated

In [7]:
##### ----- Create a model and add reactions ----- #####
model = Model("iCHO")
lr = []

for _, row in rxns.iterrows():
    r = Reaction(row['Reaction'])
    lr.append(r)
    
model.add_reactions(lr)
model

Name,iCHO
Memory address,15e68a830
Number of metabolites,0
Number of reactions,8189
Number of genes,0
Number of groups,0
Objective expression,0
Compartments,


In [8]:
##### ----- Add information to each one of the reactions ----- #####
for i,r in enumerate(tqdm(model.reactions)):
    r.build_reaction_from_string(rxns['Reaction Formula'][i])
    r.name = rxns['Reaction Name'][i]
    r.subsystem = rxns['Subsystem'][i]
    r.gene_reaction_rule = str(rxns['GPR_final'][i])
    r.lower_bound = float(rxns_attributes['Lower bound'][i])
    r.upper_bound = float(rxns_attributes['Upper bound'][i])
    # print(r)

  0%|          | 0/8189 [00:00<?, ?it/s]

unknown metabolite '10fthf5glu_c' created
unknown metabolite '10fthf5glu_l' created
unknown metabolite '10fthf5glu_m' created
unknown metabolite '10fthf6glu_c' created
unknown metabolite '10fthf6glu_l' created
unknown metabolite '10fthf6glu_m' created
unknown metabolite '10fthf7glu_c' created
unknown metabolite '10fthf7glu_l' created
unknown metabolite '10fthf7glu_m' created
unknown metabolite '10fthf_c' created
unknown metabolite '10fthf_l' created
unknown metabolite '10fthf_m' created
unknown metabolite '11docrtsl_c' created
unknown metabolite '11docrtsl_m' created
unknown metabolite '11docrtsl_r' created
unknown metabolite '11docrtstrn_c' created
unknown metabolite '11docrtstrn_m' created
unknown metabolite '11docrtstrn_r' created
unknown metabolite '12HPET_c' created
unknown metabolite 'atp_c' created
unknown metabolite 'h2o_c' created
unknown metabolite '12HPET_e' created
unknown metabolite 'adp_c' created
unknown metabolite 'h_c' created
unknown metabolite 'pi_c' created
unknown 

unknown metabolite 'c162_2Z_9Zcoa_m' created
unknown metabolite '3hoc161_9Zcoa_m' created
unknown metabolite 'c163_2Z_7E_10Ecoa_m' created
unknown metabolite '3hoc162_7E_10Ecoa_m' created
unknown metabolite 'c163_2Z_7Z_10Zcoa_m' created
unknown metabolite '3hoc162_7Z_10Zcoa_m' created
unknown metabolite 'c163_2Z_7Z_10Zcoa_x' created
unknown metabolite '3hoc162_7Z_10Zcoa_x' created
unknown metabolite 'c164_2Z_7Z_10Z_13Zcoa_m' created
unknown metabolite '3hoc163_7Z_10Z_13Zcoa_m' created
unknown metabolite 'c164_2Z_7Z_10Z_13Zcoa_x' created
unknown metabolite '3hoc163_7Z_10Z_13Zcoa_x' created
unknown metabolite '2hpdcacoa_m' created
unknown metabolite '3hohpdcacoa_m' created
unknown metabolite '3hoctaACP_c' created
unknown metabolite 'toctd2eACP_c' created
unknown metabolite 'od2coa_m' created
unknown metabolite '3hodcoa_m' created
unknown metabolite 'od2coa_x' created
unknown metabolite '3hodcoa_x' created
unknown metabolite 'ode2coa_m' created
unknown metabolite '3hodecoa_m' created
unkn

unknown metabolite '3oc161_9Zcoa_m' created
unknown metabolite '3oc162_7E_10Ecoa_m' created
unknown metabolite '3oc162_7Z_10Zcoa_m' created
unknown metabolite '3oc162_7Z_10Zcoa_x' created
unknown metabolite '3oc163_7Z_10Z_13Zcoa_m' created
unknown metabolite '3oc163_7Z_10Z_13Zcoa_x' created
unknown metabolite '3ohpdcacoa_m' created
unknown metabolite '3oodcoa_m' created
unknown metabolite '3oodcoa_x' created
unknown metabolite '3ooedcoa_m' created
unknown metabolite '3oc181_9Zcoa_m' created
unknown metabolite '3oc181_9Zcoa_x' created
unknown metabolite '3oc182_9E_12Ecoa_m' created
unknown metabolite '3oc182_9Z_12Zcoa_m' created
unknown metabolite '3oc182_9Z_12Zcoa_x' created
unknown metabolite '3oc183_9Z_12Z_15Zcoa_m' created
unknown metabolite '3oc183_9Z_12Z_15Zcoa_x' created
unknown metabolite '3oc183_6Z_9Z_12Zcoa_m' created
unknown metabolite '3oc183_6Z_9Z_12Zcoa_x' created
unknown metabolite '3oc184_6Z_9Z_12Z_15Zcoa_m' created
unknown metabolite '3oc184_6Z_9Z_12Z_15Zcoa_x' created


unknown metabolite '5hoxnfkyn_c' created
unknown metabolite '5htrp_e' created
unknown metabolite '5hxkyn_c' created
unknown metabolite '5hxkynam_c' created
unknown metabolite '46dhoxquin_c' created
unknown metabolite '5mta_e' created
unknown metabolite '5mta_c' created
unknown metabolite '5mthf_e' created
unknown metabolite '5mthf_c' created
unknown metabolite '5oxpro_e' created
unknown metabolite '5oxpro_c' created
unknown metabolite '5thf_c' created
unknown metabolite '5thf_l' created
unknown metabolite '5thf_m' created
unknown metabolite '6dhf_c' created
unknown metabolite '6dhf_l' created
unknown metabolite '6dhf_m' created
unknown metabolite '6htststerone_c' created
unknown metabolite '6htststerone_e' created
unknown metabolite '6htststerone_r' created
unknown metabolite '6thf_c' created
unknown metabolite '6thf_l' created
unknown metabolite '6thf_m' created
unknown metabolite '7dhchsterol_r' created
unknown metabolite '7dhchsterol_c' created
unknown metabolite '7dhf_c' created
un

...

unknown metabolite 'CE4898_c' created
unknown metabolite 'CE4898_r' created
unknown metabolite 'CE1925_r' created
unknown metabolite 'CE5853_r' created
unknown metabolite 'CE1926_r' created
unknown metabolite 'CE5854_r' created
unknown metabolite 'dsmsterol_c' created
unknown metabolite 'CE4968_m' created
unknown metabolite 'CE4970_m' created
unknown metabolite 'CE4969_m' created
unknown metabolite 'CE5860_c' created
unknown metabolite 'CE2095_c' created
unknown metabolite 'CE3086_c' created
unknown metabolite 'CE3092_c' created
unknown metabolite 'CE2095_m' created
unknown metabolite 'CE3092_m' created
unknown metabolite 'CE5829_c' created
unknown metabolite 'CE5865_c' created
unknown metabolite 'CE5866_c' created
unknown metabolite 'CE5867_c' created
unknown metabolite 'CE5868_c' created
unknown metabolite 'CE5869_c' created
unknown metabolite 'CE5016_c' created
unknown metabolite 'CE5875_m' created
unknown metabolite 'CE5068_m' created
unknown metabolite 'CE2102_c' created
unknown m

unknown metabolite 'zymstnl_c' created
unknown metabolite 'CE6027_c' created
unknown metabolite 'CE6027_m' created
unknown metabolite 'CE4830_c' created
unknown metabolite 'CE4837_c' created
unknown metabolite 'CE4830_r' created
unknown metabolite 'CE4837_r' created
unknown metabolite 'CE4833_c' created
unknown metabolite 'CE4831_c' created
unknown metabolite 'CE4835_c' created
unknown metabolite 'CE4834_c' created
unknown metabolite 'CE4849_c' created
unknown metabolite 'CE4834_r' created
unknown metabolite 'CE4849_r' created
unknown metabolite 'CE4841_c' created
unknown metabolite 'CE4840_c' created
unknown metabolite 'CE4842_c' created
unknown metabolite 'CE4843_c' created
unknown metabolite 'CE4845_c' created
unknown metabolite 'CE4844_c' created
unknown metabolite 'CE4846_c' created
unknown metabolite 'CE4847_c' created
unknown metabolite 'CE4852_c' created
unknown metabolite 'CE4854_c' created
unknown metabolite 'CE4850_c' created
unknown metabolite 'CE4848_c' created
unknown met

unknown metabolite 'CE7091_x' created
unknown metabolite 'CE7082_c' created
unknown metabolite 'CE7083_c' created
unknown metabolite 'CE7084_c' created
unknown metabolite 'CE7085_c' created
unknown metabolite 'CE7086_c' created
unknown metabolite 'CE7087_c' created
unknown metabolite 'CE7088_c' created
unknown metabolite 'HC02213_c' created
unknown metabolite 'CE7088_n' created
unknown metabolite 'HC02213_n' created
unknown metabolite 'CE7081_n' created
unknown metabolite 'CE7109_n' created
unknown metabolite 'CE7081_r' created
unknown metabolite 'CE7109_r' created
unknown metabolite 'CE7110_c' created
unknown metabolite 'CE7114_c' created
unknown metabolite 'CE7109_c' created
unknown metabolite 'CE7113_c' created
unknown metabolite 'CE7113_r' created
unknown metabolite 'CE7109_x' created
unknown metabolite 'CE7113_x' created
unknown metabolite 'CE7112_c' created
unknown metabolite 'CE7112_r' created
unknown metabolite 'CE7112_x' created
unknown metabolite 'CE7111_c' created
unknown me

unknown metabolite 's4l2n2bdl4fn4m2masn_g' created
unknown metabolite 's4l2n2bdl4n4m2masn_g' created
unknown metabolite 's4l3n3l4fn4m2masn_g' created
unknown metabolite 's4l3n3l4n4m2masn_g' created
unknown metabolite 's2l2n2m2masn_g' created
unknown metabolite 's3l3n3m2masn_g' created
unknown metabolite 's3l3n3rm2masn_g' created
unknown metabolite 's3l3fn3m2masn_g' created
unknown metabolite 's3l3fn3rm2masn_g' created
unknown metabolite 's4l4fn4m2masn_g' created
unknown metabolite 's3lnl3n3m2masn_g' created
unknown metabolite 's4l4n4m2masn_g' created
unknown metabolite 's2l2fn2m2masn_g' created
unknown metabolite 'cs_d_pre5_g' created
unknown metabolite 'cs_b_pre5_g' created
unknown metabolite 'hs_pre12_g' created
unknown metabolite 'hs_pre13_g' created
unknown metabolite 'hs_pre14_g' created
unknown metabolite 'hs_pre15_g' created
unknown metabolite 'hs_deg11_l' created
unknown metabolite 'cs_e_pre5a_g' created
unknown metabolite 'cs_e_pre5b_g' created
unknown metabolite 'cs_e_deg1_l'

unknown metabolite 'glyald_m' created
unknown metabolite 'r5p_r' created
unknown metabolite 'HC01162_c' created
unknown metabolite 'HC00342_m' created
unknown metabolite 'glyc_R_m' created
unknown metabolite '3pg_m' created
unknown metabolite '3hpp_m' created
unknown metabolite 'HC01161_c' created
unknown metabolite 'HC00682_m' created
unknown metabolite 'HC00695_m' created
unknown metabolite 'HC01377_m' created
unknown metabolite 'HC01115_c' created
unknown metabolite 'HC01254_c' created
unknown metabolite 'HC00361_c' created
unknown metabolite 'HC10859_r' created
unknown metabolite 'HC10859_x' created
unknown metabolite 'pmtcrn_r' created
unknown metabolite 'pmtcrn_x' created
unknown metabolite 'lnlccoa_r' created
unknown metabolite 'lnlccrn_r' created
unknown metabolite 'lnlccrn_x' created
unknown metabolite 'arachdcrn_r' created
unknown metabolite 'arachdcrn_x' created
unknown metabolite 'hdcecrn_r' created
unknown metabolite 'HC01712_m' created
unknown metabolite 'HC00576_c' creat

In [9]:
##### ----- Add information for each metabolite ----- #####
metabolites_dict = metabolites.set_index('BiGG ID').to_dict('dict')
metabolites_dict['Name']

for met in model.metabolites:
    met.name = metabolites_dict['Name'][f'{met}']
    met.formula = metabolites_dict['Formula'][f'{met}']
    met.compartment = metabolites_dict['Compartment'][f'{met}'].split(' - ')[0]
    
model

Name,iCHO
Memory address,15e68a830
Number of metabolites,5431
Number of reactions,8189
Number of genes,1503
Number of groups,0
Objective expression,0
Compartments,"c, l, m, r, e, x, n, g, , im"


In [27]:
##### ----- Build the S matrix ----- #####
'''
num_metabolites = len(model.metabolites)
num_reactions = len(model.reactions)
S = np.zeros((num_metabolites, num_reactions))
for i, metabolite in enumerate(model.metabolites):
    for j, reaction in enumerate(model.reactions):
        S[i, j] = reaction.metabolites.get(metabolite.id, 0)
'''

S = cobra.util.create_stoichiometric_matrix(model, array_type='dense')

In [28]:
S

array([[-1.,  1.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0., -1.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0., -2.],
       [ 0.,  0.,  0., ...,  0.,  0.,  2.]])

In [54]:
##### ----- Save the model ----- #####
write_sbml_model(model, "iCHOv2.xml")

### Blocked reactions and Dead-Ends

In [25]:
##### ----- FVA ----- #####
import importlib
# from utils import runMinMax_GF
import utils
importlib.reload(utils)
model.solver = 'gurobi'


for rxn in model.reactions:
    rxn.bounds = -1000, 1000
minmax = utils.runMinMax_Single(model, end_rxn_index=None)

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000


In [30]:
import importlib
import utils
importlib.reload(utils)
dead_ends = utils.detect_dead_ends(S)

# Detect dead-end metabolites
is_dead_end = utils.detect_dead_ends(S)

# Get the indices of the dead-end metabolites
dead_end_indices = np.nonzero(is_dead_end)[0]

# Get the names of the dead-end metabolites
dead_end_metabolites = [model.metabolites[i].id for i in dead_end_indices]
print(f'Number of dead end metabolites: {len(dead_end_metabolites)}')

# Print the names of the dead-end metabolites
with open('Dead_ends.txt', 'w') as f:
    for dead_met in dead_end_metabolites:
        print(dead_met, file=f)

Number of dead end metabolites: 2110


In [31]:
##### Print Reactions with the min and max fluxes #####
with open('FVA_Results.txt', 'w') as f:
    for i, j in enumerate(minmax):
        print(model.reactions[i].id, "Min: ",j[0], "Max: ", j[1], file=f)
        
##### Print Blocked Reactions #####
with open('Blocked_Reactions.txt', 'w') as f:
    for i, j in enumerate(minmax):
        if j[0] == 0 and j[1] == 0:
            print(model.reactions[i].id, "Min: ",j[0], "Max: ", j[1], file=f)

In [32]:
# Check Mass Balance
for rxn in model.reactions:
    rxn.check_mass_balance()

In [13]:
# Initiliase model
for rxn_exchange in model.exchanges:
    rxn_exchange.bounds = (-1000, 1000)
    
bio_id = model.reactions.index('biomass_producing')
atp_id = model.reactions.index('DM_atp_c')

model.reactions[bio_id].bounds = (0, 100)
model.objective = 'biomass_producing'
model.objective = 'DM_atp_c'
model.optimize()

(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000

In [149]:
with open('Biomass_Metabolites_Reactions.txt', 'w') as f:
    bio_mets = model.reactions.get_by_id('biomass_producing').metabolites
    for bio_met in bio_mets:
        f.write("Metabolite: " + bio_met.name + "\n")
        f.write("Reactions: " + ", ".join([r.name for r in bio_met.reactions]) + "\n")
        f.write("Formula: " + ", ".join([r.build_reaction_string() for r in bio_met.reactions]) + "\n\n")

# Detect all the excahnge reactions that have no uptake
from cobra.flux_analysis import flux_variability_analysis
FVA_Result = flux_variability_analysis(model, model.exchanges)

FVA_Result = a[a.minimum == 0]
with open('FVA_Excange.txt', 'w') as f:
    f.write("reaction\tminimum\tmaximum\n")
    for index, row in FVA_Result.iterrows():
        f.write(f"{index}\t{row['minimum']}\t{row['maximum']}\n")

In [33]:
for reaction in model.reactions:
    with model as model:
        reaction.knock_out()
        model.optimize()
        print('%s blocked (bounds: %s), new growth rate %f' %
              (reaction.id, str(reaction.bounds), model.objective.value))

10FTHF5GLUtl blocked (bounds: (0, 0)), new growth rate 0.000000
10FTHF5GLUtm blocked (bounds: (0, 0)), new growth rate 0.000000
10FTHF6GLUtl blocked (bounds: (0, 0)), new growth rate 0.000000
10FTHF6GLUtm blocked (bounds: (0, 0)), new growth rate 0.000000
10FTHF7GLUtl blocked (bounds: (0, 0)), new growth rate 0.000000
10FTHF7GLUtm blocked (bounds: (0, 0)), new growth rate 0.000000
10FTHFtl blocked (bounds: (0, 0)), new growth rate 0.000000
10FTHFtm blocked (bounds: (0, 0)), new growth rate 0.000000
11DOCRTSLtm blocked (bounds: (0, 0)), new growth rate 0.000000
11DOCRTSLtr blocked (bounds: (0, 0)), new growth rate 0.000000
11DOCRTSTRNtm blocked (bounds: (0, 0)), new growth rate 0.000000
11DOCRTSTRNtr blocked (bounds: (0, 0)), new growth rate 0.000000
12HPETATP blocked (bounds: (0, 0)), new growth rate 0.000000
12HPETUPKt blocked (bounds: (0, 0)), new growth rate 0.000000
13DAMPPOX blocked (bounds: (0, 0)), new growth rate 0.000000
15HPETATP blocked (bounds: (0, 0)), new growth rate 0.00

3HAD201n9p blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD203n6m blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD203n6p blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD204n3m blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD204n3p blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD204n6m blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD204n6p blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD205n3m blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD205n3p blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD220p blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD221n9p blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD224n6m blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD224n6p blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD2251n3m blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD2251n3p blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD226n3m blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD226n

3HDH70m blocked (bounds: (0, 0)), new growth rate 0.000000
3HDH80m blocked (bounds: (0, 0)), new growth rate 0.000000
3HDH80p blocked (bounds: (0, 0)), new growth rate 0.000000
3HDH81n3m blocked (bounds: (0, 0)), new growth rate 0.000000
3HDH81n3p blocked (bounds: (0, 0)), new growth rate 0.000000
3HDH90m blocked (bounds: (0, 0)), new growth rate 0.000000
3HDHDMHPm blocked (bounds: (0, 0)), new growth rate 0.000000
3HDHDMNm blocked (bounds: (0, 0)), new growth rate 0.000000
3HDHMPm blocked (bounds: (0, 0)), new growth rate 0.000000
3HDHPRISTp blocked (bounds: (0, 0)), new growth rate 0.000000
3HDHTMHDp blocked (bounds: (0, 0)), new growth rate 0.000000
3HDHTMTDp blocked (bounds: (0, 0)), new growth rate 0.000000
3HKYNAKGAT blocked (bounds: (0, 0)), new growth rate 0.000000
3HKYNAKGATm blocked (bounds: (0, 0)), new growth rate 0.000000
3HLYTCL blocked (bounds: (0, 0)), new growth rate 0.000000
3HPRODHx blocked (bounds: (0, 0)), new growth rate 0.000000
3HXKYNDCL blocked (bounds: (0, 0))

AASAD3m blocked (bounds: (0, 0)), new growth rate 0.000000
AATAi blocked (bounds: (0, 0)), new growth rate 0.000000
ABC_TCDBd blocked (bounds: (0, 0)), new growth rate 0.000000
ABO1g blocked (bounds: (0, 0)), new growth rate 0.000000
ABO2g blocked (bounds: (0, 0)), new growth rate 0.000000
ABO3g blocked (bounds: (0, 0)), new growth rate 0.000000
ABO4g blocked (bounds: (0, 0)), new growth rate 0.000000
ABO5g blocked (bounds: (0, 0)), new growth rate 0.000000
ABO6g blocked (bounds: (0, 0)), new growth rate 0.000000
ABO7g blocked (bounds: (0, 0)), new growth rate 0.000000
ABO8g blocked (bounds: (0, 0)), new growth rate 0.000000
ABO9g blocked (bounds: (0, 0)), new growth rate 0.000000
ABTArm blocked (bounds: (0, 0)), new growth rate 0.000000
ABTD blocked (bounds: (0, 0)), new growth rate 0.000000
ABTti blocked (bounds: (0, 0)), new growth rate 0.000000
ABUTD blocked (bounds: (0, 0)), new growth rate 0.000000
ABUTDm blocked (bounds: (0, 0)), new growth rate 0.000000
ABUTt2r blocked (bounds:

ACGALK blocked (bounds: (0, 0)), new growth rate 0.000000
ACGALK2 blocked (bounds: (0, 0)), new growth rate 0.000000
ACGALtlg blocked (bounds: (0, 0)), new growth rate 0.000000
ACGALtly blocked (bounds: (0, 0)), new growth rate 0.000000
ACGAM2E blocked (bounds: (0, 0)), new growth rate 0.000000
ACGAM6PSi blocked (bounds: (0, 0)), new growth rate 0.000000
ACGAMK blocked (bounds: (0, 0)), new growth rate 0.000000
ACGAMPM blocked (bounds: (0, 0)), new growth rate 0.000000
ACGAMtg blocked (bounds: (0, 0)), new growth rate 0.000000
ACGAMtl blocked (bounds: (0, 0)), new growth rate 0.000000
ACGAMtly blocked (bounds: (0, 0)), new growth rate 0.000000
ACGBGBSIDEtg blocked (bounds: (0, 0)), new growth rate 0.000000
ACGBGBSIDEtl blocked (bounds: (0, 0)), new growth rate 0.000000
ACGLUte blocked (bounds: (0, 0)), new growth rate 0.000000
ACGLUtm blocked (bounds: (0, 0)), new growth rate 0.000000
ACGPID blocked (bounds: (0, 0)), new growth rate 0.000000
ACGSm blocked (bounds: (0, 0)), new growth r

...

r1711 blocked (bounds: (0, 0)), new growth rate 0.000000
r1713 blocked (bounds: (0, 0)), new growth rate 0.000000
r1715 blocked (bounds: (0, 0)), new growth rate 0.000000
r1716 blocked (bounds: (0, 0)), new growth rate 0.000000
r1717 blocked (bounds: (0, 0)), new growth rate 0.000000
r1720 blocked (bounds: (0, 0)), new growth rate 0.000000
r1721 blocked (bounds: (0, 0)), new growth rate 0.000000
r1722 blocked (bounds: (0, 0)), new growth rate 0.000000
r1723 blocked (bounds: (0, 0)), new growth rate 0.000000
r1725 blocked (bounds: (0, 0)), new growth rate 0.000000
r1726 blocked (bounds: (0, 0)), new growth rate 0.000000
r1728 blocked (bounds: (0, 0)), new growth rate 0.000000
r1730 blocked (bounds: (0, 0)), new growth rate 0.000000
r1731 blocked (bounds: (0, 0)), new growth rate 0.000000
r1732 blocked (bounds: (0, 0)), new growth rate 0.000000
r1733 blocked (bounds: (0, 0)), new growth rate 0.000000
r1734 blocked (bounds: (0, 0)), new growth rate 0.000000
r1735 blocked (bounds: (0, 0)),

r1876 blocked (bounds: (0, 0)), new growth rate 0.000000
r1877 blocked (bounds: (0, 0)), new growth rate 0.000000
r1879 blocked (bounds: (0, 0)), new growth rate 0.000000
r1880 blocked (bounds: (0, 0)), new growth rate 0.000000
r1881 blocked (bounds: (0, 0)), new growth rate 0.000000
r1882 blocked (bounds: (0, 0)), new growth rate 0.000000
r1883 blocked (bounds: (0, 0)), new growth rate 0.000000
r1884 blocked (bounds: (0, 0)), new growth rate 0.000000
r1885 blocked (bounds: (0, 0)), new growth rate 0.000000
r1886 blocked (bounds: (0, 0)), new growth rate 0.000000
r1887 blocked (bounds: (0, 0)), new growth rate 0.000000
r1888 blocked (bounds: (0, 0)), new growth rate 0.000000
r1889 blocked (bounds: (0, 0)), new growth rate 0.000000
r1890 blocked (bounds: (0, 0)), new growth rate 0.000000
r1891 blocked (bounds: (0, 0)), new growth rate 0.000000
r1892 blocked (bounds: (0, 0)), new growth rate 0.000000
r1893 blocked (bounds: (0, 0)), new growth rate 0.000000
r1894 blocked (bounds: (0, 0)),

r2024 blocked (bounds: (0, 0)), new growth rate 0.000000
r2026 blocked (bounds: (0, 0)), new growth rate 0.000000
r2027 blocked (bounds: (0, 0)), new growth rate 0.000000
r2028 blocked (bounds: (0, 0)), new growth rate 0.000000
r2029 blocked (bounds: (0, 0)), new growth rate 0.000000
r2030 blocked (bounds: (0, 0)), new growth rate 0.000000
r2031 blocked (bounds: (0, 0)), new growth rate 0.000000
r2032 blocked (bounds: (0, 0)), new growth rate 0.000000
r2034 blocked (bounds: (0, 0)), new growth rate 0.000000
r2035 blocked (bounds: (0, 0)), new growth rate 0.000000
r2036 blocked (bounds: (0, 0)), new growth rate 0.000000
r2037 blocked (bounds: (0, 0)), new growth rate 0.000000
r2038 blocked (bounds: (0, 0)), new growth rate 0.000000
r2039 blocked (bounds: (0, 0)), new growth rate 0.000000
r2040 blocked (bounds: (0, 0)), new growth rate 0.000000
r2041 blocked (bounds: (0, 0)), new growth rate 0.000000
r2042 blocked (bounds: (0, 0)), new growth rate 0.000000
r2043 blocked (bounds: (0, 0)),

r2179 blocked (bounds: (0, 0)), new growth rate 0.000000
r2180 blocked (bounds: (0, 0)), new growth rate 0.000000
r2181 blocked (bounds: (0, 0)), new growth rate 0.000000
r2182 blocked (bounds: (0, 0)), new growth rate 0.000000
r2183 blocked (bounds: (0, 0)), new growth rate 0.000000
r2184 blocked (bounds: (0, 0)), new growth rate 0.000000
r2185 blocked (bounds: (0, 0)), new growth rate 0.000000
r2186 blocked (bounds: (0, 0)), new growth rate 0.000000
r2187 blocked (bounds: (0, 0)), new growth rate 0.000000
r2188 blocked (bounds: (0, 0)), new growth rate 0.000000
r2189 blocked (bounds: (0, 0)), new growth rate 0.000000
r2190 blocked (bounds: (0, 0)), new growth rate 0.000000
r2191 blocked (bounds: (0, 0)), new growth rate 0.000000
r2192 blocked (bounds: (0, 0)), new growth rate 0.000000
r2193 blocked (bounds: (0, 0)), new growth rate 0.000000
r2194 blocked (bounds: (0, 0)), new growth rate 0.000000
r2195 blocked (bounds: (0, 0)), new growth rate 0.000000
r2196 blocked (bounds: (0, 0)),

r2323 blocked (bounds: (0, 0)), new growth rate 0.000000
r2324 blocked (bounds: (0, 0)), new growth rate 0.000000
r2325 blocked (bounds: (0, 0)), new growth rate 0.000000
r2326 blocked (bounds: (0, 0)), new growth rate 0.000000
r2327 blocked (bounds: (0, 0)), new growth rate 0.000000
r2328 blocked (bounds: (0, 0)), new growth rate 0.000000
r2329 blocked (bounds: (0, 0)), new growth rate 0.000000
r2330 blocked (bounds: (0, 0)), new growth rate 0.000000
r2331 blocked (bounds: (0, 0)), new growth rate 0.000000
r2332 blocked (bounds: (0, 0)), new growth rate 0.000000
r2333 blocked (bounds: (0, 0)), new growth rate 0.000000
r2334 blocked (bounds: (0, 0)), new growth rate 0.000000
r2335 blocked (bounds: (0, 0)), new growth rate 0.000000
r2338 blocked (bounds: (0, 0)), new growth rate 0.000000
r2341 blocked (bounds: (0, 0)), new growth rate 0.000000
r2342 blocked (bounds: (0, 0)), new growth rate 0.000000
r2343 blocked (bounds: (0, 0)), new growth rate 0.000000
r2344 blocked (bounds: (0, 0)),

In [34]:
import tempfile
from pprint import pprint
from cobra.io import write_sbml_model, validate_sbml_model
with tempfile.NamedTemporaryFile(suffix='.xml') as f_sbml:
    write_sbml_model(model, filename=f_sbml.name)
    report = validate_sbml_model(filename=f_sbml.name)

pprint(report)

(None,
 {'COBRA_CHECK': [],
  'COBRA_ERROR': ["Required attribute 'id' cannot be found or parsed in "
                  "'<Compartment>'."],
  'COBRA_FATAL': [],
  'COBRA_WARNING': [],
  'SBML_ERROR': ['E0 (Error): SBML component consistency (core, L22); Invalid '
                 'attribute found on Compartment object; A Compartment object '
                 "must have the required attributes 'id' and 'constant', and "
                 "may have the optional attributes 'metaid', 'sboTerm', "
                 "'name', 'spatialDimensions', 'size' and 'units'. No other "
                 'attributes from the SBML Level 3 Core namespace are '
                 'permitted on a Compartment object.\n'
                 'Reference: L3V1 Section 4.5\n'
                 " The required attribute 'id' is missing.\n",
                 'E1 (Error): SBML component consistency (fbc, L682); Chemical '
                 'formula must be string; The value of attribute '
                 "'fbc:chemicalFormu